In [ ]:

#### The data in the step below is cming from the feature engineering code

# =========================================
# STEP 0: Imports
# =========================================
from google.colab import files
import pandas as pd
import numpy as np
# =========================================
# STEP 1: Upload Files
# =========================================
print("📂 Please upload these files:")


uploaded = files.upload()  # <-- Upload all four files here

daily_sales_with_percentage = pd.read_csv("daily_sales_with_percentage.csv", low_memory=False,parse_dates=['date'],dayfirst=True)

# Fill NaN in 'department_id' with -1 after loading
if 'department_id' in daily_sales_with_percentage.columns:
    daily_sales_with_percentage['department_id'] = daily_sales_with_percentage['department_id'].fillna(-1)

# Make negative values in the 'y' column positive
if 'y' in daily_sales_with_percentage.columns:
    daily_sales_with_percentage['y'] = daily_sales_with_percentage['y'].apply(lambda x: x * -1 if x < 0 else x)


print("\n✅ Files uploaded and loaded successfully.")
display(daily_sales_with_percentage.head())

📂 Please upload these files:


In [ ]:
###### Check data upload below if uploaded successfully

In [ ]:
# Get unique combinations of location_id, sales_type_id, and department_id
unique_combinations = daily_sales_with_percentage[['location_id', 'sales_type_id', 'department_id']].drop_duplicates()

print("✅ Unique combinations of location_id, sales_type_id, and department_id:")
display(unique_combinations)

✅ Unique combinations of location_id, sales_type_id, and department_id:


,location_id,sales_type_id,department_id
0,45093,45287,-1.0
52705,121752,244513,22316.0
105410,121752,244513,22313.0
158115,121752,244513,22315.0
210820,121766,244513,22313.0
263525,121766,244513,22315.0
316230,121766,244513,22316.0


In [ ]:
#### Add Feature to be Predicted - 15 min sales value at department level

In [ ]:
# Calculate the total daily sales for each location and sales type
daily_total_sales_location_st = daily_sales_with_percentage.groupby(
    ['date', 'location_id', 'sales_type_id']
)['y'].transform('sum')

# Calculate the percentage of sales for each department at a given time slot
# Removed multiplication by 100 to make it a proportion
daily_sales_with_percentage['y_pred_department_st'] = np.where(
    daily_total_sales_location_st != 0,
    (daily_sales_with_percentage['y'] / daily_total_sales_location_st),
    0 # Use 0 for cases where total daily sales are zero
)

print("✅ Added 'y_pred_department_st' column to daily_sales_with_percentage as a proportion.")
display(daily_sales_with_percentage.head())

✅ Added 'y_pred_department_st' column to daily_sales_with_percentage as a proportion.


,ds,location_id,sales_type_id,department_id,corporation_id,postal_code,y,real_feel,precipitation,snow,...,opening_time,closing_time,is_open,halo_type,offset_days,is_holiday_halo,halo_offset_days,daily_total_sales,y_percentage_15min,y_pred_department_st
0,2024-01-01 00:00:00,45093,45287,-1.0,46113,T7Z 2W7,22.5,0.71,0.0,0.0,...,11:00:00,00:00:00,0,holiday,0.0,1,0.0,6173.51,0.003645,0.003645
1,2024-01-01 00:15:00,45093,45287,-1.0,46113,T7Z 2W7,0.0,0.71,0.0,0.0,...,11:00:00,00:00:00,0,holiday,0.0,1,0.0,6173.51,0.000000,0.000000
2,2024-01-01 00:30:00,45093,45287,-1.0,46113,T7Z 2W7,0.0,0.71,0.0,0.0,...,11:00:00,00:00:00,0,holiday,0.0,1,0.0,6173.51,0.000000,0.000000
3,2024-01-01 00:45:00,45093,45287,-1.0,46113,T7Z 2W7,0.0,0.71,0.0,0.0,...,11:00:00,00:00:00,0,holiday,0.0,1,0.0,6173.51,0.000000,0.000000
4,2024-01-01 01:00:00,45093,45287,-1.0,46113,T7Z 2W7,0.0,-0.49,0.0,0.0,...,11:00:00,00:00:00,0,holiday,0.0,1,0.0,6173.51,0.000000,0.000000


In [ ]:
df=daily_sales_with_percentage.copy()

In [ ]:
# Step 1: department-level daily sales
dept_daily_sales = (
    df.groupby(["date", "location_id", "sales_type_id", "department_id"], as_index=False)
      .agg({"y": "sum"})
      .rename(columns={"y": "dept_daily_sales"})
)

# Step 2: total daily sales (across all departments)
total_daily_sales = (
    dept_daily_sales.groupby(["date", "location_id", "sales_type_id"], as_index=False)
    .agg({"dept_daily_sales": "sum"})
    .rename(columns={"dept_daily_sales": "daily_total_sales"})
)

# Step 3: merge back
dept_daily_sales = dept_daily_sales.merge(
    total_daily_sales,
    on=["date", "location_id", "sales_type_id"],
    how="left"
)

# Step 4: compute department share
dept_daily_sales["y_pred_department"] = (
    dept_daily_sales["dept_daily_sales"] / dept_daily_sales["daily_total_sales"]
)

# Validation: sums should all be 1
check = dept_daily_sales.groupby(["date", "location_id", "sales_type_id"])["y_pred_department"].sum()
print("Should all be 1:", check.unique())


Should all be 1: [1. 1. 1. 1. 0.]


In [ ]:
# Save to CSV
dept_daily_sales.to_csv("dept_daily_sales.csv", index=False)
print("✅ dept_daily_sales saved to dept_daily_sales.csv")

✅ dept_daily_sales saved to dept_daily_sales.csv


In [ ]:
# dept_daily_sales already has:
# ['date', 'location_id', 'sales_type_id', 'department_id', 'dept_daily_sales', 'daily_total_sales', 'y_pred_department']

# Merge back to original df
df = df.merge(
    dept_daily_sales[["date", "location_id", "sales_type_id", "department_id", "y_pred_department"]],
    on=["date", "location_id", "sales_type_id", "department_id"],
    how="left"
)

# Optional: fill NaN with 0 for days/departments with no sales
df["y_pred_department"] = df["y_pred_department"].fillna(0)

# Quick check
check = df.groupby(["date", "location_id", "sales_type_id"])["y_pred_department"].sum().reset_index()
print("After merge, sums per day-location-sales_type (should be 1 or 0):")
print(check[check["y_pred_department"] != 1].head(10))


After merge, sums per day-location-sales_type (should be 1 or 0):
         date  location_id  sales_type_id  y_pred_department
0  2024-01-01        45093          45287               96.0
1  2024-01-01       121752         244513               96.0
2  2024-01-01       121766         244513               96.0
3  2024-01-02        45093          45287               96.0
4  2024-01-02       121752         244513               96.0
5  2024-01-02       121766         244513               96.0
6  2024-01-03        45093          45287               96.0
7  2024-01-03       121752         244513               96.0
8  2024-01-03       121766         244513               96.0
9  2024-01-04        45093          45287               96.0


In [ ]:
### Checked sampes fr 1st Jan - variabe y_pred_department_st is crrect

In [ ]:
# =========================================
# STEP 0: Imports
# =========================================
from google.colab import files
import pandas as pd
import numpy as np

# =========================================
# STEP 1: Upload Files
# =========================================
print("📂 Please upload these files:")
print("1. daily_forecast_2025.csv (columns: location_id, date, y_pred, department_id - if available)")
print("2. actuals_15min.csv (columns: location_id, ds_x, y)")
print("3. locations.csv (columns: id, corporation_id, postal_code)")
print("4. weather_hourly.csv (columns: id, country_code, ds, postal_code, real_feel, precipitation, snow, coverage, weather_code, icon)")


uploaded = files.upload()  # <-- Upload all four files here

# Read daily_forecast with the correct date format
daily_forecast = pd.read_csv(
    "forecast_2025_xgb_final.csv",
    low_memory=False,
    parse_dates=['date'],
    dayfirst=True  # Specify that the day comes first (DD-MM-YYYY)
)
#actuals_15min_2024 = pd.read_csv("aggregated_15min_df_2024.csv",low_memory=False, parse_dates=['ds'],dayfirst=True)
#actuals_15min_2025 = pd.read_csv("aggregated_15min_df_2025.csv",low_memory=False, parse_dates=['ds'],dayfirst=True)
locations = pd.read_csv("locations.csv")
weather = pd.read_csv("weather_hourly.csv", low_memory=False, parse_dates=['ds'],dayfirst=True)

# Ensure y_pred in daily_forecast is not negative
if 'y_pred' in daily_forecast.columns:
    negative_y_pred_count = (daily_forecast['y_pred'] < 0).sum()
    if negative_y_pred_count > 0:
        print(f"\n❗ Warning: Found {negative_y_pred_count} negative values in 'y_pred' column of daily_forecast. Making them positive.")
        daily_forecast['y_pred'] = daily_forecast['y_pred'].apply(lambda x: x * -1 if x < 0 else x)


print("\n✅ Files uploaded and loaded successfully.")
display(daily_forecast.head())
display(locations.head())
# display(weather.head())

# Check if 'department_id' is in daily_forecast and add a note if not
if 'department_id' not in daily_forecast.columns:
    print("\n❗ Note: 'department_id' column was not found in 'forecast_2025_xgb_final.csv'. The subsequent steps depending on this column might fail.")

📂 Please upload these files:
1. daily_forecast_2025.csv (columns: location_id, date, y_pred, department_id - if available)
2. actuals_15min.csv (columns: location_id, ds_x, y)
3. locations.csv (columns: id, corporation_id, postal_code)
4. weather_hourly.csv (columns: id, country_code, ds, postal_code, real_feel, precipitation, snow, coverage, weather_code, icon)


Saving forecast_2025_xgb_final.csv to forecast_2025_xgb_final.csv
Saving locations.csv to locations.csv
Saving weather_hourly.csv to weather_hourly.csv

❗ Warning: Found 2 negative values in 'y_pred' column of daily_forecast. Making them positive.

✅ Files uploaded and loaded successfully.


/tmp/ipython-input-1522889097.py:30: UserWarning: Parsing dates in %Y-%m-%d %H:%M:%S format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  weather = pd.read_csv("weather_hourly.csv", low_memory=False, parse_dates=['ds'],dayfirst=True)


,date,location_id,sales_type_id,y_daily,holiday,is_holiday,industry,real_feel,precipitation,snow,...,offset_days,is_holiday_halo,halo_offset_days,day_of_month,week_of_year,quarter,month,week_of_month,day_of_week_numeric,y_pred
0,2025-01-01,121752,244513,3242.80,10,1,1,3.693750,0.00,0.000,...,0,1,0,1,1,1,1,1,2,3688.9856
1,2025-01-02,121752,244513,3511.74,-1,0,1,6.071667,0.00,0.000,...,0,0,0,2,1,1,1,1,3,4099.4067
2,2025-01-03,121752,244513,5262.18,-1,0,1,3.169167,0.00,0.000,...,0,0,0,3,1,1,1,1,4,5505.9155
3,2025-01-04,121752,244513,3813.45,-1,0,1,6.986667,0.00,0.000,...,0,0,0,4,1,1,1,1,5,4644.6436
4,2025-01-05,121752,244513,3104.90,-1,0,1,-1.806250,1.71,0.472,...,0,0,0,5,1,1,1,1,6,3927.1543


,id,corporation_id,postal_code
0,144484,91906,66209
1,144391,91906,66220
2,144818,92250,92651
3,144789,92250,92618
4,144338,91908,10002



❗ Note: 'department_id' column was not found in 'forecast_2025_xgb_final.csv'. The subsequent steps depending on this column might fail.


### Uploading the forecasts from Prophet model for to be compared with lightgbm for comparsion

In [ ]:
# =========================================
# STEP 0: Imports
# =========================================
from google.colab import files
import pandas as pd
import numpy as np

# =========================================
# STEP 1: Upload Files
# =========================================
print("📂 Please upload these files:")


uploaded = files.upload()  # <-- Upload all four files here

aggregated_15min_df_forecasts = pd.read_csv("df_aggregated_future.csv", low_memory=False,parse_dates=['ds'],dayfirst=True)


print("\n✅ Files uploaded and loaded successfully.")
display(aggregated_15min_df_forecasts.head())

📂 Please upload these files:


Saving df_aggregated_future.csv to df_aggregated_future.csv

✅ Files uploaded and loaded successfully.


,ds,location_id,sales_type_id,department_id,corporation_id,postal_code,y
0,2024-01-01 00:00:00,121752,244513,22315.0,78415,74063,0.0
1,2024-01-01 00:15:00,121752,244513,22315.0,78415,74063,0.0
2,2024-01-01 00:30:00,121752,244513,22315.0,78415,74063,0.0
3,2024-01-01 00:45:00,121752,244513,22315.0,78415,74063,0.0
4,2024-01-01 01:00:00,121752,244513,22315.0,78415,74063,0.0


In [ ]:
# Fill NaN in 'department_id' with -1 for location_id = 45093 in aggregated_15min_df_forecasts
aggregated_15min_df_forecasts.loc[
    (aggregated_15min_df_forecasts['location_id'] == 45093) & (aggregated_15min_df_forecasts['department_id'].isna()),
    'department_id'
] = -1

print("✅ Updated 'department_id' to -1 for NaN values at location_id 45093 in aggregated_15min_df_forecasts.")
display(aggregated_15min_df_forecasts.head())

✅ Updated 'department_id' to -1 for NaN values at location_id 45093 in aggregated_15min_df_forecasts.


,ds,location_id,sales_type_id,department_id,corporation_id,postal_code,y
0,2024-01-01 00:00:00,121752,244513,22315.0,78415,74063,0.0
1,2024-01-01 00:15:00,121752,244513,22315.0,78415,74063,0.0
2,2024-01-01 00:30:00,121752,244513,22315.0,78415,74063,0.0
3,2024-01-01 00:45:00,121752,244513,22315.0,78415,74063,0.0
4,2024-01-01 01:00:00,121752,244513,22315.0,78415,74063,0.0


**Reasoning**:
Save the comparison dataframe to a CSV file as requested in the subtask.



In [ ]:
import pandas as pd
import numpy as np
from lightgbm import LGBMRegressor

# ==========================
# ENSURE y_pred_department IS PRESENT
# Merge y_pred_department from dept_daily_sales into df if not already present
if 'y_pred_department' not in df.columns:
    print("Merging y_pred_department into df...")
    # Assuming dept_daily_sales is available and has the 'y_pred_department' column
    # Re-run the merge logic from cell 2ndTkncH10nf
    df = df.merge(
        dept_daily_sales[["date", "location_id", "sales_type_id", "department_id", "y_pred_department"]],
        on=["date", "location_id", "sales_type_id", "department_id"],
        how="left"
    )
    # Fill NaN after merging
    df["y_pred_department"] = df["y_pred_department"].fillna(0)
    print("Merge complete.")

# ==========================
# LOAD AND CLEAN DATA
# Ensure df has date column as datetime
df["date"] = pd.to_datetime(df["date"], errors="coerce")
df["hour"] = df["time"].str.slice(0, 2).astype(int)
df["minute"] = df["time"].str.slice(3, 5).astype(int)
df["slot_id"] = df["hour"] * 4 + df["minute"] // 15

# Cyclic encoding
df["hour_sin"] = np.sin(2 * np.pi * df["hour"] / 24)
df["hour_cos"] = np.cos(2 * np.pi * df["hour"] / 24)
df["minute_sin"] = np.sin(2 * np.pi * df["minute"] / 60)
df["minute_cos"] = np.cos(2 * np.pi * df["minute"] / 60)

# Drop unwanted columns - ensure y_pred_department is NOT dropped if it exists
drop_cols = ["weather_code", "opening_time", "closing_time", "corporation_id", "postal_code",
             "y", "ds", "y_percentage_15min", "daily_total_sales"]
df = df.drop(columns=[c for c in drop_cols if c in df.columns and c != 'y_pred_department'])


# Numeric conversion
for col in ["real_feel", "precipitation", "snow", "coverage"]:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors="coerce")

# Fill missing department_id
if 'department_id' in df.columns:
    df['department_id'] = df['department_id'].fillna(-1)

# Encode categorical
categorical_cols = ["holiday", "industry", "day", "halo_type", "is_open", "is_holiday_halo"]
for col in categorical_cols:
    if col in df.columns:
        df[col] = pd.factorize(df[col])[0]


# ==========================
# SPLIT TRAIN / TEST
df_2024 = df[df["date"].dt.year == 2024].copy()
df_2025 = df[df["date"].dt.year == 2025].copy()

target = "y_pred_department_st"
id_cols = ["date", "location_id", "sales_type_id", "department_id", "time"]
# y_pred_department will be used for identifying high/low share departments, not as a direct feature
exclude_cols = id_cols + [target, "y_pred_department"]
X_cols = [c for c in df_2024.columns if c not in exclude_cols]


# ==========================
# CREATE TEMPORAL FEATURES & SMOOTH TARGETS FOR TRAINING DATA (2024)
df_2024 = df_2024.sort_values(["location_id", "sales_type_id", "department_id", "slot_id"])

# Calculate smoothed target (y_pred_department_st)
df_2024['y_smoothed'] = df_2024.groupby(["location_id","sales_type_id","department_id"])[target]\
                                .transform(lambda x: x.rolling(2, min_periods=1).mean())

# Lag features for the smoothed target
df_2024['prev_slot'] = df_2024.groupby(["location_id","sales_type_id","department_id"])['y_smoothed'].shift(1).fillna(0)

# Features for LGBM model
X_train = df_2024[X_cols + ['prev_slot']] # Add lagged feature to training features
y_train = df_2024['y_smoothed'] # Use smoothed target for training

# ==========================
# IDENTIFY HIGH-SHARE VS LOW-SHARE DEPARTMENTS
# Compute average daily department share from original y_pred_department column
avg_dept_share = df_2024.groupby(['location_id', 'sales_type_id', 'department_id'])['y_pred_department'].mean().reset_index()
high_share_threshold = 0.20  # Example threshold
high_share_departments = avg_dept_share[avg_dept_share['y_pred_department'] >= high_share_threshold]
low_share_departments = avg_dept_share[avg_dept_share['y_pred_department'] < high_share_threshold]

print(f"High share departments identified (threshold >= {high_share_threshold}):")
display(high_share_departments)
print("\nLow share departments identified:")
display(low_share_departments)


# ==========================
# LIGHTGBM MODEL FOR HIGH-SHARE
model = LGBMRegressor(
    n_estimators=1000,
    learning_rate=0.03,
    max_depth=5,
    subsample=0.9,
    colsample_bytree=0.8,
    random_state=42
)
# Filter training data for high-share departments
X_train_high = df_2024[df_2024.set_index(['location_id', 'sales_type_id', 'department_id']).index.isin(
    high_share_departments.set_index(['location_id','sales_type_id','department_id']).index
)][X_cols + ['prev_slot']] # Include lagged feature in high-share training features
y_train_high = df_2024[df_2024.set_index(['location_id', 'sales_type_id', 'department_id']).index.isin(
    high_share_departments.set_index(['location_id','sales_type_id','department_id']).index
)]['y_smoothed'] # Use smoothed target for high-share training

# Check if X_train_high is empty before fitting
if not X_train_high.empty:
    model.fit(X_train_high, y_train_high)
else:
    print("Warning: No training data for high-share departments. Skipping LGBM training.")


# ==========================
# PREDICT SLOT PERCENTAGES FOR 2025
df_2025 = df_2025.sort_values(["location_id", "sales_type_id", "department_id", "slot_id"])
df_2025['y_percentage_15min_pred'] = 0.0 # Initialize with float type

# Add lagged feature to 2025 data (initialized with 0 for the first slot of each group)
df_2025['prev_slot'] = df_2025.groupby(["location_id","sales_type_id","department_id"])['y_percentage_15min_pred'].shift(1).fillna(0)


mask_open = df_2025["is_open"] == 1

# Create mask for high-share departments in 2025 data
high_mask_2025 = df_2025.set_index(['location_id', 'sales_type_id', 'department_id']).index.isin(
    high_share_departments.set_index(['location_id','sales_type_id','department_id']).index
)

# Apply LGBM only for high-share and open stores
lgbm_mask = mask_open & high_mask_2025
if not df_2025.loc[lgbm_mask, X_cols + ['prev_slot']].empty: # Check if DataFrame slice is not empty, include lagged feature
    df_2025.loc[lgbm_mask, "y_percentage_15min_pred"] = model.predict(df_2025.loc[lgbm_mask, X_cols + ['prev_slot']]) # Predict using lagged feature
else:
    print("Warning: No data points for high-share departments when stores are open in 2025. Predictions set to 0 for these.")


# ==========================
# PD-STYLE FOR LOW-SHARE
# Compute historical slot proportions from 2024 for low-share departments
slot_props_2024_low = df_2024[df_2024.set_index(['location_id', 'sales_type_id', 'department_id']).index.isin(
    low_share_departments.set_index(['location_id','sales_type_id','department_id']).index
)].copy()

# Check if slot_props_2024_low is empty before proceeding with PD-style
if slot_props_2024_low.empty:
    print("Warning: No historical data available for low-share departments. Skipping PD-style forecasting for low-share departments.")
else:
    # Calculate the sum of y_pred_department_st for each combination of location, sales type, department, and slot
    slot_props_2024_low_sum = slot_props_2024_low.groupby(['location_id','sales_type_id','department_id','slot_id'])['y_pred_department_st'].sum().reset_index()

    # Calculate the total sum of y_pred_department_st for each combination of location, sales type, and department
    total_slot_sum = slot_props_2024_low_sum.groupby(['location_id','sales_type_id','department_id'])['y_pred_department_st'].sum().reset_index(name='total_sum')

    # Merge to get the total sum for normalization
    slot_props_2024_low_agg = slot_props_2024_low_sum.merge(
        total_slot_sum,
        on=['location_id','sales_type_id','department_id'],
        how='left'
    )

    # Calculate the slot percentage, handling division by zero
    slot_props_2024_low_agg['slot_percentage'] = np.where(
        slot_props_2024_low_agg['total_sum'] > 0,
        slot_props_2024_low_agg['y_pred_department_st'] / slot_props_2024_low_agg['total_sum'],
        0
    )

    slot_props_2024_low_agg = slot_props_2024_low_agg[['location_id','sales_type_id','department_id','slot_id','slot_percentage']]


    # Create mask for low-share departments in 2025 data
    low_mask_2025 = df_2025.set_index(['location_id', 'sales_type_id', 'department_id']).index.isin(
        low_share_departments.set_index(['location_id','sales_type_id','department_id']).index
    )

    # Apply PD-style only for low-share and open stores
    pd_mask = mask_open & low_mask_2025

    print(f"\nShape of df_2025.loc[pd_mask]: {df_2025.loc[pd_mask].shape}")
    print(f"Shape of slot_props_2024_low_agg: {slot_props_2024_low_agg.shape}")

    if not df_2025.loc[pd_mask].empty and not slot_props_2024_low_agg.empty: # Check if DataFrame slice is not empty and historical data exists
        low_dept_2025 = df_2025.loc[pd_mask].merge(
            slot_props_2024_low_agg,
            on=['location_id','sales_type_id','department_id','slot_id'],
            how='left'
        )
        # Fill NaN slot_percentage with 0 if no historical data for a slot
        low_dept_2025['slot_percentage'] = low_dept_2025['slot_percentage'].fillna(0)
        df_2025.loc[pd_mask, 'y_percentage_15min_pred'] = low_dept_2025['slot_percentage'].values
    else:
         print("Warning: No data points for low-share departments when stores are open in 2025 or no historical data available for low-share departments (after aggregation). Predictions set to 0 for these.")


# ==========================
# NORMALIZE WITHIN DEPARTMENT
def normalize_within_department(group):
    total = group.loc[group["is_open"] == 1, "y_percentage_15min_pred"].sum()
    if total > 0:
        group.loc[group["is_open"] == 1, "y_percentage_15min_pred"] /= total
    group["y_percentage_15min_pred"] = group["y_percentage_15min_pred"].clip(0,1)
    return group

df_2025 = df_2025.groupby(['date','location_id','sales_type_id','department_id'], group_keys=False).apply(normalize_within_department)

# ==========================
# MERGE WITH DAILY FORECAST
daily_forecast_new = daily_forecast.copy()[["date","location_id","sales_type_id","y_pred"]]
daily_forecast_new["date"] = pd.to_datetime(daily_forecast_new["date"], errors='coerce')

slot_forecast = df_2025.merge(
    daily_forecast_new,
    on=["date","location_id","sales_type_id"],
    how="left"
)

slot_forecast["raw_y_15min_forecast"] = slot_forecast["y_pred"] * slot_forecast["y_percentage_15min_pred"]

# Normalize across departments
def normalize_across_departments(group):
    total_forecast = group["raw_y_15min_forecast"].sum()
    y_pred_total = group["y_pred"].iloc[0]
    if total_forecast > 0:
        group["y_15min_forecast"] = group["raw_y_15min_forecast"] * y_pred_total / total_forecast
    else:
        group["y_15min_forecast"] = 0
    return group

slot_forecast = slot_forecast.groupby(["date","location_id","sales_type_id"], group_keys=False).apply(normalize_across_departments)
slot_forecast["y_15min_forecast"] = slot_forecast["y_15min_forecast"].clip(0,None)

# ==========================
# FINAL OUTPUT
final_output_lightgbm = slot_forecast[["date","location_id","sales_type_id","department_id","time","y_15min_forecast"]].copy()
print("✅ Finished HYBRID slot-level forecast (LGBM + PD-style)")
print(final_output_lightgbm.head())

High share departments identified (threshold >= 0.2):


,location_id,sales_type_id,department_id,y_pred_department
0,45093,45287,-1.0,0.997268
1,121752,244513,22313.0,0.278180
2,121752,244513,22315.0,0.259771
3,121752,244513,22316.0,0.456585
5,121766,244513,22315.0,0.301941
6,121766,244513,22316.0,0.502102



Low share departments identified:


,location_id,sales_type_id,department_id,y_pred_department
4,121766,244513,22313.0,0.190493


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.021446 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1328
[LightGBM] [Info] Number of data points in the train set: 210816, number of used features: 20
[LightGBM] [Info] Start training from score 0.004854
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain

/tmp/ipython-input-631589271.py:214: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_2025 = df_2025.groupby(['date','location_id','sales_type_id','department_id'], group_keys=False).apply(normalize_within_department)


✅ Finished HYBRID slot-level forecast (LGBM + PD-style)
        date  location_id  sales_type_id  department_id      time  \
0 2025-01-01        45093          45287           -1.0  00:00:00   
1 2025-01-02        45093          45287           -1.0  00:00:00   
2 2025-01-03        45093          45287           -1.0  00:00:00   
3 2025-01-04        45093          45287           -1.0  00:00:00   
4 2025-01-05        45093          45287           -1.0  00:00:00   

   y_15min_forecast  
0          0.000000  
1          0.000000  
2         31.400737  
3         34.107050  
4          0.000000  


/tmp/ipython-input-631589271.py:239: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  slot_forecast = slot_forecast.groupby(["date","location_id","sales_type_id"], group_keys=False).apply(normalize_across_departments)


In [ ]:
# Rename the 'y' column in aggregated_15min_df_forecasts to make it distinct
aggregated_15min_df_forecasts_renamed = aggregated_15min_df_forecasts.rename(columns={'y': 'y_pred_previous_model'})

# Select the relevant columns from the actuals data
actuals_df = daily_sales_with_percentage[['ds', 'location_id', 'sales_type_id', 'department_id', 'y']].copy()

# Rename the 'y' column in actuals_df
actuals_df = actuals_df.rename(columns={'y': 'y_actual'})

# Ensure the datetime columns are consistent for merging
actuals_df['ds'] = pd.to_datetime(actuals_df['ds'])
aggregated_15min_df_forecasts_renamed['ds'] = pd.to_datetime(aggregated_15min_df_forecasts_renamed['ds'])
final_output_lightgbm['date'] = pd.to_datetime(final_output_lightgbm['date'])

# Merge the actuals and previous model predictions
comparison_df = actuals_df.merge(
    aggregated_15min_df_forecasts_renamed[['ds', 'location_id', 'sales_type_id', 'department_id', 'y_pred_previous_model']],
    on=['ds', 'location_id', 'sales_type_id', 'department_id'],
    how='left'
)

# Merge with the new LightGBM forecasts
# Need to create a 'ds' column in final_output_lightgbm for merging
final_output_lightgbm['ds'] = pd.to_datetime(final_output_lightgbm['date'].dt.strftime('%Y-%m-%d') + ' ' + final_output_lightgbm['time'])

comparison_df = comparison_df.merge(
    final_output_lightgbm[['ds', 'location_id', 'sales_type_id', 'department_id', 'y_15min_forecast']],
    on=['ds', 'location_id', 'sales_type_id', 'department_id'],
    how='left'
)

# Filter for January 2025
comparison_df_jan_2025 = comparison_df[(comparison_df['ds'].dt.year == 2025) & (comparison_df['ds'].dt.month == 1)].copy()

# Calculate Absolute Percentage Error (APE) at the row level, setting to NaN if y_actual is 0
comparison_df_jan_2025['APE_previous_model'] = np.where(
    comparison_df_jan_2025['y_actual'] != 0,
    np.abs((comparison_df_jan_2025['y_actual'] - comparison_df_jan_2025['y_pred_previous_model']) / comparison_df_jan_2025['y_actual']) * 100,
    np.nan
)

comparison_df_jan_2025['APE_lightgbm_model'] = np.where(
    comparison_df_jan_2025['y_actual'] != 0,
    np.abs((comparison_df_jan_2025['y_actual'] - comparison_df_jan_2025['y_15min_forecast']) / comparison_df_jan_2025['y_actual']) * 100,
    np.nan
)

# Aggregate by sales_type_id, department_id, and location_id, taking the mean of APE
aggregated_comparison_mape = comparison_df_jan_2025.groupby(
    ['sales_type_id', 'department_id', 'location_id']
).agg(
    MAPE_previous_model=('APE_previous_model', 'mean'),
    MAPE_lightgbm_model=('APE_lightgbm_model', 'mean')
).reset_index()


# Display the aggregated comparison table (MAPE)
print("Aggregated Comparison for January 2025 (Sales Type, Department, Location) - MAPE (NaN for zero actuals):")
display(aggregated_comparison_mape)

Aggregated Comparison for January 2025 (Sales Type, Department, Location) - MAPE (NaN for zero actuals):


,sales_type_id,department_id,location_id,MAPE_previous_model,MAPE_lightgbm_model
0,45287,-1.0,45093,159.137926,82.950469
1,244513,22313.0,121752,85.436244,125.854788
2,244513,22313.0,121766,80.004529,132.631178
3,244513,22315.0,121752,67.001465,65.789250
4,244513,22315.0,121766,181.599831,132.029328
5,244513,22316.0,121752,86.348981,83.133025
6,244513,22316.0,121766,149.989568,104.136794


In [ ]:
# Save the comparison_df_jan_2025 DataFrame to a CSV file
comparison_df_jan_2025.to_csv('comparison_df_jan_2025.csv', index=False)

print("✅ comparison_df_jan_2025 DataFrame saved to 'comparison_df_jan_2025.csv'")

✅ comparison_df_jan_2025 DataFrame saved to 'comparison_df_jan_2025.csv'


In [ ]:
# Calculate the overall average APE for each model
overall_avg_ape_previous = comparison_df_jan_2025['APE_previous_model'].mean()
overall_avg_ape_lightgbm = comparison_df_jan_2025['APE_lightgbm_model'].mean()

print(f"Overall Average APE (Previous Model): {overall_avg_ape_previous:.2f}")
print(f"Overall Average APE (LightGBM Model): {overall_avg_ape_lightgbm:.2f}")

Overall Average APE (Previous Model): 122.56
Overall Average APE (LightGBM Model): 101.16


In [ ]:
import pandas as pd
import numpy as np
from lightgbm import LGBMRegressor

# ==========================
# ENSURE y_pred_department IS PRESENT
# ==========================
if 'y_pred_department' not in df.columns:
    print("Merging y_pred_department into df...")
    df = df.merge(
        dept_daily_sales[["date", "location_id", "sales_type_id", "department_id", "y_pred_department"]],
        on=["date", "location_id", "sales_type_id", "department_id"],
        how="left"
    )
    df["y_pred_department"] = df["y_pred_department"].fillna(0)
    print("Merge complete.")

# ==========================
# LOAD AND CLEAN DATA
# ==========================
df["date"] = pd.to_datetime(df["date"], errors="coerce")
df["hour"] = df["time"].str.slice(0, 2).astype(int)
df["minute"] = df["time"].str.slice(3, 5).astype(int)
df["slot_id"] = df["hour"] * 4 + df["minute"] // 15

# Cyclic encoding
df["hour_sin"] = np.sin(2 * np.pi * df["hour"] / 24)
df["hour_cos"] = np.cos(2 * np.pi * df["hour"] / 24)
df["minute_sin"] = np.sin(2 * np.pi * df["minute"] / 60)
df["minute_cos"] = np.cos(2 * np.pi * df["minute"] / 60)

# Drop unwanted columns except y_pred_department
drop_cols = ["weather_code", "opening_time", "closing_time", "corporation_id", "postal_code",
             "y", "ds", "y_percentage_15min", "daily_total_sales"]
df = df.drop(columns=[c for c in drop_cols if c in df.columns and c != 'y_pred_department'])

# Numeric conversion
for col in ["real_feel", "precipitation", "snow", "coverage"]:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors="coerce")

# Fill missing department_id
df['department_id'] = df['department_id'].fillna(-1)

# Encode categorical features
categorical_cols = ["holiday", "industry", "day", "halo_type", "is_open", "is_holiday_halo"]
for col in categorical_cols:
    if col in df.columns:
        df[col] = pd.factorize(df[col])[0]

# ==========================
# SPLIT TRAIN / TEST
# ==========================
df_2024 = df[df["date"].dt.year == 2024].copy()
df_2025 = df[df["date"].dt.year == 2025].copy()

target = "y_pred_department_st"
id_cols = ["date", "location_id", "sales_type_id", "department_id", "time"]
exclude_cols = [target, "y_pred_department", "date", "time"]  # exclude target + raw date/time
X_cols = [c for c in df_2024.columns if c not in exclude_cols]

# Make IDs categorical for LightGBM
for col in ["location_id", "sales_type_id", "department_id"]:
    df_2024[col] = df_2024[col].astype("int32")
    df_2025[col] = df_2025[col].astype("int32")

# ==========================
# CREATE TEMPORAL FEATURES
# ==========================
df_2024 = df_2024.sort_values(["location_id","sales_type_id","department_id","slot_id"])
df_2024['y_smoothed'] = df_2024.groupby(["location_id","sales_type_id","department_id"])[target]\
                               .transform(lambda x: x.rolling(2, min_periods=1).mean())
df_2024['prev_slot'] = df_2024.groupby(["location_id","sales_type_id","department_id"])['y_smoothed']\
                              .shift(1).fillna(0)

X_train = df_2024[X_cols + ['prev_slot']]
y_train = df_2024['y_smoothed']

# ==========================
# LIGHTGBM MODEL
# ==========================
categorical_features = ["location_id", "sales_type_id", "department_id"]

model = LGBMRegressor(
    n_estimators=1000,
    learning_rate=0.03,
    max_depth=8,
    subsample=0.9,
    colsample_bytree=0.8,
    random_state=42
)

model.fit(
    X_train, y_train,
    categorical_feature=[c for c in categorical_features if c in X_train.columns]
)

# ==========================
# PREDICT FOR 2025
# ==========================
df_2025 = df_2025.sort_values(["location_id","sales_type_id","department_id","slot_id"])
df_2025['prev_slot'] = df_2025.groupby(["location_id","sales_type_id","department_id"])['y_pred_department'].shift(1).fillna(0)

df_2025['y_percentage_15min_pred'] = model.predict(df_2025[X_cols + ['prev_slot']])

# ==========================
# NORMALIZE WITHIN DEPARTMENT
# ==========================
def normalize_within_department(group):
    total = group.loc[group["is_open"] == 1, "y_percentage_15min_pred"].sum()
    if total > 0:
        group.loc[group["is_open"] == 1, "y_percentage_15min_pred"] /= total
    group["y_percentage_15min_pred"] = group["y_percentage_15min_pred"].clip(0,1)
    return group

df_2025 = df_2025.groupby(['date','location_id','sales_type_id','department_id'], group_keys=False).apply(normalize_within_department)

# ==========================
# MERGE WITH DAILY FORECAST
# ==========================
daily_forecast_new = daily_forecast.copy()[["date","location_id","sales_type_id","y_pred"]]
daily_forecast_new["date"] = pd.to_datetime(daily_forecast_new["date"], errors='coerce')

slot_forecast = df_2025.merge(
    daily_forecast_new,
    on=["date","location_id","sales_type_id"],
    how="left"
)

slot_forecast["raw_y_15min_forecast"] = slot_forecast["y_pred"] * slot_forecast["y_percentage_15min_pred"]

# Normalize across departments
def normalize_across_departments(group):
    total_forecast = group["raw_y_15min_forecast"].sum()
    y_pred_total = group["y_pred"].iloc[0]
    if total_forecast > 0:
        group["y_15min_forecast"] = group["raw_y_15min_forecast"] * y_pred_total / total_forecast
    else:
        group["y_15min_forecast"] = 0
    return group

slot_forecast = slot_forecast.groupby(["date","location_id","sales_type_id"], group_keys=False).apply(normalize_across_departments)
slot_forecast["y_15min_forecast"] = slot_forecast["y_15min_forecast"].clip(0,None)

# ==========================
# FINAL OUTPUT
# ==========================
final_output_lightgbm = slot_forecast[["date","location_id","sales_type_id","department_id","time","y_15min_forecast"]].copy()
print("✅ Finished GLOBAL LightGBM slot-level forecast")
print(final_output_lightgbm.head())


[LightGBM] [Warning] Met categorical feature which contains sparse values. Consider renumbering to consecutive integers started from zero
[LightGBM] [Warning] Met categorical feature which contains sparse values. Consider renumbering to consecutive integers started from zero
[LightGBM] [Warning] Met negative value in categorical features, will convert it to NaN
[LightGBM] [Warning] Met categorical feature which contains sparse values. Consider renumbering to consecutive integers started from zero
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.050438 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1334
[LightGBM] [Info] Number of data points in the train set: 245952, number of used features: 23
[LightGBM] [Info] Start training from score 0.004444


/tmp/ipython-input-622787201.py:116: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_2025 = df_2025.groupby(['date','location_id','sales_type_id','department_id'], group_keys=False).apply(normalize_within_department)


✅ Finished GLOBAL LightGBM slot-level forecast
        date  location_id  sales_type_id  department_id      time  \
0 2025-01-01        45093          45287             -1  00:00:00   
1 2025-01-02        45093          45287             -1  00:00:00   
2 2025-01-03        45093          45287             -1  00:00:00   
3 2025-01-04        45093          45287             -1  00:00:00   
4 2025-01-05        45093          45287             -1  00:00:00   

   y_15min_forecast  
0          3.323914  
1        121.525525  
2         52.052669  
3         68.540204  
4        134.885044  


/tmp/ipython-input-622787201.py:142: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  slot_forecast = slot_forecast.groupby(["date","location_id","sales_type_id"], group_keys=False).apply(normalize_across_departments)


In [ ]:
import pandas as pd
import numpy as np
from lightgbm import LGBMRegressor

# ==========================
# ENSURE y_pred_department IS PRESENT
# ==========================
if 'y_pred_department' not in df.columns:
    print("Merging y_pred_department into df...")
    df = df.merge(
        dept_daily_sales[["date", "location_id", "sales_type_id", "department_id", "y_pred_department"]],
        on=["date", "location_id", "sales_type_id", "department_id"],
        how="left"
    )
    df["y_pred_department"] = df["y_pred_department"].fillna(0)
    print("Merge complete.")

# ==========================
# LOAD AND CLEAN DATA
# ==========================
df["date"] = pd.to_datetime(df["date"], errors="coerce")
df["hour"] = df["time"].str.slice(0, 2).astype(int)
df["minute"] = df["time"].str.slice(3, 5).astype(int)
df["slot_id"] = df["hour"] * 4 + df["minute"] // 15

# Cyclic encoding
df["hour_sin"] = np.sin(2 * np.pi * df["hour"] / 24)
df["hour_cos"] = np.cos(2 * np.pi * df["hour"] / 24)
df["minute_sin"] = np.sin(2 * np.pi * df["minute"] / 60)
df["minute_cos"] = np.cos(2 * np.pi * df["minute"] / 60)

# Drop unwanted columns except y_pred_department
drop_cols = ["weather_code", "opening_time", "closing_time", "corporation_id", "postal_code",
             "y", "ds", "y_percentage_15min", "daily_total_sales"]
df = df.drop(columns=[c for c in drop_cols if c in df.columns and c != 'y_pred_department'])

# Numeric conversion
for col in ["real_feel", "precipitation", "snow", "coverage"]:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors="coerce")

# Fill missing department_id
df['department_id'] = df['department_id'].fillna(-1)

# Encode categorical features
categorical_cols = ["holiday", "industry", "day", "halo_type", "is_open", "is_holiday_halo"]
for col in categorical_cols:
    if col in df.columns:
        df[col] = pd.factorize(df[col])[0]

# ==========================
# SPLIT TRAIN / TEST
# ==========================
df_2024 = df[df["date"].dt.year == 2024].copy()
df_2025 = df[df["date"].dt.year == 2025].copy()

target = "y_pred_department_st"
exclude_cols = [target, "y_pred_department", "date", "time"]  # exclude target + raw date/time
X_cols = [c for c in df_2024.columns if c not in exclude_cols]

# Make IDs categorical for LightGBM
for col in ["location_id", "sales_type_id", "department_id"]:
    df_2024[col] = df_2024[col].astype("int32")
    df_2025[col] = df_2025[col].astype("int32")

# ==========================
# TRAIN LIGHTGBM WITHOUT PREV_SLOT
# ==========================
X_train = df_2024[X_cols]
y_train = df_2024[target]

categorical_features = ["location_id", "sales_type_id", "department_id"]

model = LGBMRegressor(
    n_estimators=1000,
    learning_rate=0.03,
    max_depth=8,
    subsample=0.9,
    colsample_bytree=0.8,
    random_state=42
)

model.fit(
    X_train, y_train,
    categorical_feature=[c for c in categorical_features if c in X_train.columns]
)

# ==========================
# PREDICT FOR 2025
# ==========================
df_2025 = df_2025.sort_values(["location_id","sales_type_id","department_id","slot_id"])
df_2025['y_percentage_15min_pred'] = model.predict(df_2025[X_cols])

# ==========================
# NORMALIZE WITHIN DEPARTMENT
# ==========================
def normalize_within_department(group):
    total = group.loc[group["is_open"] == 1, "y_percentage_15min_pred"].sum()
    if total > 0:
        group.loc[group["is_open"] == 1, "y_percentage_15min_pred"] /= total
    group["y_percentage_15min_pred"] = group["y_percentage_15min_pred"].clip(0,1)
    return group

df_2025 = df_2025.groupby(['date','location_id','sales_type_id','department_id'], group_keys=False).apply(normalize_within_department)

# ==========================
# MERGE WITH DAILY FORECAST
# ==========================
daily_forecast_new = daily_forecast.copy()[["date","location_id","sales_type_id","y_pred"]]
daily_forecast_new["date"] = pd.to_datetime(daily_forecast_new["date"], errors='coerce')

slot_forecast = df_2025.merge(
    daily_forecast_new,
    on=["date","location_id","sales_type_id"],
    how="left"
)

slot_forecast["raw_y_15min_forecast"] = slot_forecast["y_pred"] * slot_forecast["y_percentage_15min_pred"]

# Normalize across departments
def normalize_across_departments(group):
    total_forecast = group["raw_y_15min_forecast"].sum()
    y_pred_total = group["y_pred"].iloc[0]
    if total_forecast > 0:
        group["y_15min_forecast"] = group["raw_y_15min_forecast"] * y_pred_total / total_forecast
    else:
        group["y_15min_forecast"] = 0
    return group

slot_forecast = slot_forecast.groupby(["date","location_id","sales_type_id"], group_keys=False).apply(normalize_across_departments)
slot_forecast["y_15min_forecast"] = slot_forecast["y_15min_forecast"].clip(0,None)

# ==========================
# FINAL OUTPUT
# ==========================
final_output_lightgbm = slot_forecast[["date","location_id","sales_type_id","department_id","time","y_15min_forecast"]].copy()
print("✅ Finished GLOBAL LightGBM slot-level forecast")
print(final_output_lightgbm.head())


[LightGBM] [Warning] Met categorical feature which contains sparse values. Consider renumbering to consecutive integers started from zero
[LightGBM] [Warning] Met categorical feature which contains sparse values. Consider renumbering to consecutive integers started from zero
[LightGBM] [Warning] Met negative value in categorical features, will convert it to NaN
[LightGBM] [Warning] Met categorical feature which contains sparse values. Consider renumbering to consecutive integers started from zero
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.039475 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1079
[LightGBM] [Info] Number of data points in the train set: 245952, number of used features: 22
[LightGBM] [Info] Start training from score 0.004444
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warn

/tmp/ipython-input-3578462443.py:104: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_2025 = df_2025.groupby(['date','location_id','sales_type_id','department_id'], group_keys=False).apply(normalize_within_department)


✅ Finished GLOBAL LightGBM slot-level forecast
        date  location_id  sales_type_id  department_id      time  \
0 2025-01-01        45093          45287             -1  00:00:00   
1 2025-01-02        45093          45287             -1  00:00:00   
2 2025-01-03        45093          45287             -1  00:00:00   
3 2025-01-04        45093          45287             -1  00:00:00   
4 2025-01-05        45093          45287             -1  00:00:00   

   y_15min_forecast  
0         37.150017  
1         22.522762  
2          4.019268  
3          0.000000  
4         55.577202  


/tmp/ipython-input-3578462443.py:130: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  slot_forecast = slot_forecast.groupby(["date","location_id","sales_type_id"], group_keys=False).apply(normalize_across_departments)


In [ ]:
# Rename the 'y' column in aggregated_15min_df_forecasts to make it distinct
aggregated_15min_df_forecasts_renamed = aggregated_15min_df_forecasts.rename(columns={'y': 'y_pred_previous_model'})

# Select the relevant columns from the actuals data
actuals_df = daily_sales_with_percentage[['ds', 'location_id', 'sales_type_id', 'department_id', 'y']].copy()

# Rename the 'y' column in actuals_df
actuals_df = actuals_df.rename(columns={'y': 'y_actual'})

# Ensure the datetime columns are consistent for merging
actuals_df['ds'] = pd.to_datetime(actuals_df['ds'])
aggregated_15min_df_forecasts_renamed['ds'] = pd.to_datetime(aggregated_15min_df_forecasts_renamed['ds'])
final_output_lightgbm['date'] = pd.to_datetime(final_output_lightgbm['date'])

# Merge the actuals and previous model predictions
comparison_df = actuals_df.merge(
    aggregated_15min_df_forecasts_renamed[['ds', 'location_id', 'sales_type_id', 'department_id', 'y_pred_previous_model']],
    on=['ds', 'location_id', 'sales_type_id', 'department_id'],
    how='left'
)

# Merge with the new LightGBM forecasts
# Need to create a 'ds' column in final_output_lightgbm for merging
final_output_lightgbm['ds'] = pd.to_datetime(final_output_lightgbm['date'].dt.strftime('%Y-%m-%d') + ' ' + final_output_lightgbm['time'])

comparison_df = comparison_df.merge(
    final_output_lightgbm[['ds', 'location_id', 'sales_type_id', 'department_id', 'y_15min_forecast']],
    on=['ds', 'location_id', 'sales_type_id', 'department_id'],
    how='left'
)

# Filter for January 2025
comparison_df_jan_2025 = comparison_df[(comparison_df['ds'].dt.year == 2025) & (comparison_df['ds'].dt.month == 1)].copy()

# Calculate Absolute Percentage Error (APE) at the row level, setting to NaN if y_actual is 0
comparison_df_jan_2025['APE_previous_model'] = np.where(
    comparison_df_jan_2025['y_actual'] != 0,
    np.abs((comparison_df_jan_2025['y_actual'] - comparison_df_jan_2025['y_pred_previous_model']) / comparison_df_jan_2025['y_actual']) * 100,
    np.nan
)

comparison_df_jan_2025['APE_lightgbm_model'] = np.where(
    comparison_df_jan_2025['y_actual'] != 0,
    np.abs((comparison_df_jan_2025['y_actual'] - comparison_df_jan_2025['y_15min_forecast']) / comparison_df_jan_2025['y_actual']) * 100,
    np.nan
)

# Aggregate by sales_type_id, department_id, and location_id, taking the mean of APE
aggregated_comparison_mape = comparison_df_jan_2025.groupby(
    ['sales_type_id', 'department_id', 'location_id']
).agg(
    MAPE_previous_model=('APE_previous_model', 'mean'),
    MAPE_lightgbm_model=('APE_lightgbm_model', 'mean')
).reset_index()


# Display the aggregated comparison table (MAPE)
print("Aggregated Comparison for January 2025 (Sales Type, Department, Location) - MAPE (NaN for zero actuals):")
display(aggregated_comparison_mape)

# Save the comparison_df_jan_2025 DataFrame to a CSV file
comparison_df_jan_2025.to_csv('comparison_df_jan_2025.csv', index=False)

print("✅ comparison_df_jan_2025 DataFrame saved to 'comparison_df_jan_2025.csv'")

Aggregated Comparison for January 2025 (Sales Type, Department, Location) - MAPE (NaN for zero actuals):


,sales_type_id,department_id,location_id,MAPE_previous_model,MAPE_lightgbm_model
0,45287,-1.0,45093,159.137926,84.139512
1,244513,22313.0,121752,85.436244,105.126859
2,244513,22313.0,121766,80.004529,134.416844
3,244513,22315.0,121752,67.001465,68.065023
4,244513,22315.0,121766,181.599831,170.432914
5,244513,22316.0,121752,86.348981,74.213887
6,244513,22316.0,121766,149.989568,103.726612


✅ comparison_df_jan_2025 DataFrame saved to 'comparison_df_jan_2025.csv'


In [ ]:
# Calculate the overall average APE for each model
overall_avg_ape_previous = comparison_df_jan_2025['APE_previous_model'].mean()
overall_avg_ape_lightgbm = comparison_df_jan_2025['APE_lightgbm_model'].mean()

print(f"Overall Average APE (Previous Model): {overall_avg_ape_previous:.2f}")
print(f"Overall Average APE (LightGBM Model): {overall_avg_ape_lightgbm:.2f}")

Overall Average APE (Previous Model): 122.56
Overall Average APE (LightGBM Model): 102.75


In [ ]:
import pandas as pd
import numpy as np
from lightgbm import LGBMRegressor

# ==========================
# ENSURE y_pred_department IS PRESENT
# ==========================
if 'y_pred_department' not in df.columns:
    print("Merging y_pred_department into df...")
    df = df.merge(
        dept_daily_sales[["date", "location_id", "sales_type_id", "department_id", "y_pred_department"]],
        on=["date", "location_id", "sales_type_id", "department_id"],
        how="left"
    )
    df["y_pred_department"] = df["y_pred_department"].fillna(0)
    print("Merge complete.")

# ==========================
# LOAD AND CLEAN DATA
# ==========================
df["date"] = pd.to_datetime(df["date"], errors="coerce")
df["hour"] = df["time"].str.slice(0, 2).astype(int)
df["minute"] = df["time"].str.slice(3, 5).astype(int)
df["slot_id"] = df["hour"] * 4 + df["minute"] // 15

# Cyclic encoding
df["hour_sin"] = np.sin(2 * np.pi * df["hour"] / 24)
df["hour_cos"] = np.cos(2 * np.pi * df["hour"] / 24)
df["minute_sin"] = np.sin(2 * np.pi * df["minute"] / 60)
df["minute_cos"] = np.cos(2 * np.pi * df["minute"] / 60)

# Drop unwanted columns except y_pred_department
drop_cols = ["weather_code", "opening_time", "closing_time", "corporation_id", "postal_code",
             "y", "ds", "y_percentage_15min", "daily_total_sales"]
df = df.drop(columns=[c for c in drop_cols if c in df.columns and c != 'y_pred_department'])

# Numeric conversion
for col in ["real_feel", "precipitation", "snow", "coverage"]:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors="coerce")

# Fill missing department_id
df['department_id'] = df['department_id'].fillna(-1)

# Encode categorical features
categorical_cols = ["holiday", "industry", "day", "halo_type", "is_open", "is_holiday_halo"]
for col in categorical_cols:
    if col in df.columns:
        df[col] = pd.factorize(df[col])[0]

# ==========================
# SPLIT TRAIN / TEST
# ==========================
df_2024 = df[df["date"].dt.year == 2024].copy()
df_2025 = df[df["date"].dt.year == 2025].copy()

target = "y_pred_department_st"
id_cols = ["date", "location_id", "sales_type_id", "department_id", "time"]
exclude_cols = id_cols + [target, "y_pred_department"]
X_cols = [c for c in df_2024.columns if c not in exclude_cols]

# ==========================
# CREATE TEMPORAL FEATURES & SMOOTH TARGETS FOR TRAINING DATA
# ==========================
df_2024 = df_2024.sort_values(["location_id","sales_type_id","department_id","slot_id"])
df_2024['y_smoothed'] = df_2024.groupby(["location_id","sales_type_id","department_id"])[target]\
                               .transform(lambda x: x.rolling(2, min_periods=1).mean())
df_2024['prev_slot'] = df_2024.groupby(["location_id","sales_type_id","department_id"])['y_smoothed']\
                              .shift(1).fillna(0)

X_train = df_2024[X_cols + ['prev_slot']]
y_train = df_2024['y_smoothed']

# ==========================
# IDENTIFY HIGH-SHARE AND LOW-SHARE DEPARTMENTS
# ==========================
avg_dept_share = df_2024.groupby(['location_id', 'sales_type_id', 'department_id'])['y_pred_department'].mean().reset_index()
high_share_threshold = 0.20
high_share_departments = avg_dept_share[avg_dept_share['y_pred_department'] >= high_share_threshold]
low_share_departments = avg_dept_share[avg_dept_share['y_pred_department'] < high_share_threshold]

# ==========================
# LIGHTGBM MODEL FOR HIGH-SHARE
# ==========================
model = LGBMRegressor(
    n_estimators=1000,
    learning_rate=0.03,
    max_depth=5,
    subsample=0.9,
    colsample_bytree=0.8,
    random_state=42
)

X_train_high = df_2024[df_2024.set_index(['location_id','sales_type_id','department_id']).index.isin(
    high_share_departments.set_index(['location_id','sales_type_id','department_id']).index
)][X_cols + ['prev_slot']]
y_train_high = df_2024[df_2024.set_index(['location_id','sales_type_id','department_id']).index.isin(
    high_share_departments.set_index(['location_id','sales_type_id','department_id']).index
)]['y_smoothed']

if not X_train_high.empty:
    model.fit(X_train_high, y_train_high)
else:
    print("Warning: No training data for high-share departments. Skipping LGBM training.")

# ==========================
# PREDICT SLOT PERCENTAGES FOR 2025
# ==========================
df_2025 = df_2025.sort_values(["location_id","sales_type_id","department_id","slot_id"])
df_2025['y_percentage_15min_pred'] = 0.0

# Lagged feature initialized with 0
df_2025['prev_slot'] = df_2025.groupby(["location_id","sales_type_id","department_id"])['y_percentage_15min_pred']\
                              .shift(1).fillna(0)

mask_open = df_2025["is_open"] == 1

# High-share departments
high_mask_2025 = df_2025.set_index(['location_id','sales_type_id','department_id']).index.isin(
    high_share_departments.set_index(['location_id','sales_type_id','department_id']).index
)
lgbm_mask = mask_open & high_mask_2025
if not df_2025.loc[lgbm_mask, X_cols + ['prev_slot']].empty:
    df_2025.loc[lgbm_mask, "y_percentage_15min_pred"] = model.predict(df_2025.loc[lgbm_mask, X_cols + ['prev_slot']])
else:
    print("Warning: No data points for high-share departments when open. Predictions set to 0.")

# Low-share departments: PD-style
slot_props_2024_low = df_2024[df_2024.set_index(['location_id','sales_type_id','department_id']).index.isin(
    low_share_departments.set_index(['location_id','sales_type_id','department_id']).index
)].copy()

if not slot_props_2024_low.empty:
    slot_props_2024_low_sum = slot_props_2024_low.groupby(['location_id','sales_type_id','department_id','slot_id'])[target].sum().reset_index()
    total_slot_sum = slot_props_2024_low_sum.groupby(['location_id','sales_type_id','department_id'])[target].sum().reset_index(name='total_sum')
    slot_props_2024_low_agg = slot_props_2024_low_sum.merge(total_slot_sum, on=['location_id','sales_type_id','department_id'], how='left')
    slot_props_2024_low_agg['slot_percentage'] = np.where(slot_props_2024_low_agg['total_sum'] > 0,
                                                          slot_props_2024_low_agg[target] / slot_props_2024_low_agg['total_sum'], 0)
    slot_props_2024_low_agg = slot_props_2024_low_agg[['location_id','sales_type_id','department_id','slot_id','slot_percentage']]

    low_mask_2025 = df_2025.set_index(['location_id','sales_type_id','department_id']).index.isin(
        low_share_departments.set_index(['location_id','sales_type_id','department_id']).index
    )
    pd_mask = mask_open & low_mask_2025

    if not df_2025.loc[pd_mask].empty and not slot_props_2024_low_agg.empty:
        low_dept_2025 = df_2025.loc[pd_mask].merge(
            slot_props_2024_low_agg,
            on=['location_id','sales_type_id','department_id','slot_id'],
            how='left'
        )
        low_dept_2025['slot_percentage'] = low_dept_2025['slot_percentage'].fillna(0)
        df_2025.loc[pd_mask, 'y_percentage_15min_pred'] = low_dept_2025['slot_percentage'].values

# ==========================
# NORMALIZE WITHIN DEPARTMENT
# ==========================
def normalize_within_department(group):
    total = group.loc[group["is_open"] == 1, "y_percentage_15min_pred"].sum()
    if total > 0:
        group.loc[group["is_open"] == 1, "y_percentage_15min_pred"] /= total
    group["y_percentage_15min_pred"] = group["y_percentage_15min_pred"].clip(0,1)
    return group

df_2025 = df_2025.groupby(['date','location_id','sales_type_id','department_id'], group_keys=False).apply(normalize_within_department)

# ==========================
# MERGE WITH DAILY FORECAST
# ==========================
daily_forecast_new = daily_forecast.copy()[["date","location_id","sales_type_id","y_pred"]]
daily_forecast_new["date"] = pd.to_datetime(daily_forecast_new["date"], errors='coerce')

slot_forecast = df_2025.merge(daily_forecast_new, on=["date","location_id","sales_type_id"], how="left")
slot_forecast["raw_y_15min_forecast"] = slot_forecast["y_pred"] * slot_forecast["y_percentage_15min_pred"]

# Normalize across departments
def normalize_across_departments(group):
    total_forecast = group["raw_y_15min_forecast"].sum()
    y_pred_total = group["y_pred"].iloc[0]
    if total_forecast > 0:
        group["y_15min_forecast"] = group["raw_y_15min_forecast"] * y_pred_total / total_forecast
    else:
        group["y_15min_forecast"] = 0
    return group

slot_forecast = slot_forecast.groupby(["date","location_id","sales_type_id"], group_keys=False).apply(normalize_across_departments)
slot_forecast["y_15min_forecast"] = slot_forecast["y_15min_forecast"].clip(0,None)

# ==========================
# FINAL OUTPUT
# ==========================
final_output_lightgbm = slot_forecast[["date","location_id","sales_type_id","department_id","time","y_15min_forecast"]].copy()
print("✅ Finished HYBRID slot-level forecast (LGBM + PD-style)")
print(final_output_lightgbm.head())


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.020887 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1328
[LightGBM] [Info] Number of data points in the train set: 210816, number of used features: 20
[LightGBM] [Info] Start training from score 0.004854
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain

/tmp/ipython-input-3749139983.py:165: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_2025 = df_2025.groupby(['date','location_id','sales_type_id','department_id'], group_keys=False).apply(normalize_within_department)


✅ Finished HYBRID slot-level forecast (LGBM + PD-style)
        date  location_id  sales_type_id  department_id      time  \
0 2025-01-01        45093          45287             -1  00:00:00   
1 2025-01-02        45093          45287             -1  00:00:00   
2 2025-01-03        45093          45287             -1  00:00:00   
3 2025-01-04        45093          45287             -1  00:00:00   
4 2025-01-05        45093          45287             -1  00:00:00   

   y_15min_forecast  
0          0.000000  
1          0.000000  
2         21.538094  
3         17.627758  
4          0.000000  


/tmp/ipython-input-3749139983.py:186: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  slot_forecast = slot_forecast.groupby(["date","location_id","sales_type_id"], group_keys=False).apply(normalize_across_departments)


In [ ]:
# Rename the 'y' column in aggregated_15min_df_forecasts to make it distinct
aggregated_15min_df_forecasts_renamed = aggregated_15min_df_forecasts.rename(columns={'y': 'y_pred_previous_model'})

# Select the relevant columns from the actuals data
actuals_df = daily_sales_with_percentage[['ds', 'location_id', 'sales_type_id', 'department_id', 'y']].copy()

# Rename the 'y' column in actuals_df
actuals_df = actuals_df.rename(columns={'y': 'y_actual'})

# Ensure the datetime columns are consistent for merging
actuals_df['ds'] = pd.to_datetime(actuals_df['ds'])
aggregated_15min_df_forecasts_renamed['ds'] = pd.to_datetime(aggregated_15min_df_forecasts_renamed['ds'])
final_output_lightgbm['date'] = pd.to_datetime(final_output_lightgbm['date'])

# Merge the actuals and previous model predictions
comparison_df = actuals_df.merge(
    aggregated_15min_df_forecasts_renamed[['ds', 'location_id', 'sales_type_id', 'department_id', 'y_pred_previous_model']],
    on=['ds', 'location_id', 'sales_type_id', 'department_id'],
    how='left'
)

# Merge with the new LightGBM forecasts
# Need to create a 'ds' column in final_output_lightgbm for merging
final_output_lightgbm['ds'] = pd.to_datetime(final_output_lightgbm['date'].dt.strftime('%Y-%m-%d') + ' ' + final_output_lightgbm['time'])

comparison_df = comparison_df.merge(
    final_output_lightgbm[['ds', 'location_id', 'sales_type_id', 'department_id', 'y_15min_forecast']],
    on=['ds', 'location_id', 'sales_type_id', 'department_id'],
    how='left'
)

# Filter for January 2025
comparison_df_jan_2025 = comparison_df[(comparison_df['ds'].dt.year == 2025) & (comparison_df['ds'].dt.month == 1)].copy()

# Calculate Absolute Percentage Error (APE) at the row level, setting to NaN if y_actual is 0
comparison_df_jan_2025['APE_previous_model'] = np.where(
    comparison_df_jan_2025['y_actual'] != 0,
    np.abs((comparison_df_jan_2025['y_actual'] - comparison_df_jan_2025['y_pred_previous_model']) / comparison_df_jan_2025['y_actual']) * 100,
    np.nan
)

comparison_df_jan_2025['APE_lightgbm_model'] = np.where(
    comparison_df_jan_2025['y_actual'] != 0,
    np.abs((comparison_df_jan_2025['y_actual'] - comparison_df_jan_2025['y_15min_forecast']) / comparison_df_jan_2025['y_actual']) * 100,
    np.nan
)

# Aggregate by sales_type_id, department_id, and location_id, taking the mean of APE
aggregated_comparison_mape = comparison_df_jan_2025.groupby(
    ['sales_type_id', 'department_id', 'location_id']
).agg(
    MAPE_previous_model=('APE_previous_model', 'mean'),
    MAPE_lightgbm_model=('APE_lightgbm_model', 'mean')
).reset_index()


# Display the aggregated comparison table (MAPE)
print("Aggregated Comparison for January 2025 (Sales Type, Department, Location) - MAPE (NaN for zero actuals):")
display(aggregated_comparison_mape)

# Save the comparison_df_jan_2025 DataFrame to a CSV file
comparison_df_jan_2025.to_csv('comparison_df_jan_2025.csv', index=False)

print("✅ comparison_df_jan_2025 DataFrame saved to 'comparison_df_jan_2025.csv'")

Aggregated Comparison for January 2025 (Sales Type, Department, Location) - MAPE (NaN for zero actuals):


,sales_type_id,department_id,location_id,MAPE_previous_model,MAPE_lightgbm_model
0,45287,-1.0,45093,159.137926,83.116127
1,244513,22313.0,121752,85.436244,124.703685
2,244513,22313.0,121766,80.004529,132.646485
3,244513,22315.0,121752,67.001465,65.626562
4,244513,22315.0,121766,181.599831,133.818172
5,244513,22316.0,121752,86.348981,82.708520
6,244513,22316.0,121766,149.989568,103.009602


✅ comparison_df_jan_2025 DataFrame saved to 'comparison_df_jan_2025.csv'


In [ ]:
# Calculate the overall average APE for each model
overall_avg_ape_previous = comparison_df_jan_2025['APE_previous_model'].mean()
overall_avg_ape_lightgbm = comparison_df_jan_2025['APE_lightgbm_model'].mean()

print(f"Overall Average APE (Previous Model): {overall_avg_ape_previous:.2f}")
print(f"Overall Average APE (LightGBM Model): {overall_avg_ape_lightgbm:.2f}")

Overall Average APE (Previous Model): 122.56
Overall Average APE (LightGBM Model): 101.01
